In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import datetime
import time
import re

In [2]:
chromedriver_dir = r'C:\Users\LG\빅데이터 실습\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)

In [3]:
url = 'https://tv.naver.com/jtbc.superband2/playlists'

driver.get(url)
time.sleep(3)

In [4]:
# 플레이 리스트 다 보여주기 위함
# 더보기 계속 진행
while(1):
    close = 0
    try:
        xpath = """//*[@id="cds_flick"]/div/div[2]/div/div/div/div/div[3]/a"""
        driver.find_element_by_xpath(xpath).click()
    except:
        close = 1
        print("더보기 끝")
        
    if close == 1:
        break;

더보기 끝


In [5]:
# BeautifulSoup 실행

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)

<html lang="ko"><head>
<title id="headTitle">
    	
    		네이버 TV
    		
    		
    	
    </title>
<meta charset="utf-8"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex" name="robots"/>
<meta content="summary" name="twitter:card"/>
<meta content="100012287952710" property="fb:admins"/>
<meta content="232491033870086" property="fb:app_id"/>
<meta content="https://tv.naver.com/jtbc.superband2" property="og:url"/>
<meta content="슈퍼밴드2 : 네이버TV" property="og:title"/>
<meta content="음악천재들의 수제 음악프로젝트 [슈퍼밴드]" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="https://phinf.pstatic.net/tvcast/20210618_106/lIsvN_1624009475324i3Co8_PNG/20210618_184058_592.png?type=round_232_232" property="og:image"/>
<link href="/resources/release/css/tvcast_20211209161204.css" rel="stylesheet" type="text/css"/>
<link href

In [31]:
# 저장할 데이터 프레임
superband_view = pd.DataFrame(columns=("회차", "영상 이름", "조회수", "좋아요 수"))
superband_view

,회차,영상 이름,조회수,좋아요 수


In [32]:
# 재생 목록 이름만 가져옴
# 회차에 해당하는 리스트만 가져옴
playlist_names = []
playlists = soup.find_all('div', 'playlist _MM_CARD')
i = -1
for playlist in playlists:
    playlist_name = playlist.find('h3', 'play_tit').get_text() # 재생 목록 이름 가져옴
    i += 1
    if i == 14:
        break;    # 회차 영상만 가져오만 끝나도록
    if i <= 4:
        playlist_name = playlist_name[:9].strip() # 재생 목록을 프로그램 이름 + 회차까지 그리고 공백 제거
        playlist_names.append(playlist_name)
    else:
        playlist_name = playlist_name[:8].strip() # 재생 목록을 프로그램 이름 + 회차까지 그리고 공백 제거
        playlist_names.append(playlist_name)
        
print(playlist_names)

['슈퍼밴드2 14회', '슈퍼밴드2 13회', '슈퍼밴드2 12회', '슈퍼밴드2 11회', '슈퍼밴드2 10회', '슈퍼밴드2 9회', '슈퍼밴드2 8회', '슈퍼밴드2 7회', '슈퍼밴드2 6회', '슈퍼밴드2 5회', '슈퍼밴드2 4회', '슈퍼밴드2 3회', '슈퍼밴드2 2회', '슈퍼밴드2 1회']


In [33]:
playlists = soup.find_all('div', 'playlist _MM_CARD')
# 재생 목록 이름 리스트
i = -1
for playlist in playlists:  
    i += 1
    if i == 14: # 슈퍼밴드 1~14회 영상 다 가져온 후 종료
        break
    
    
    playlist_name_list = [] # 재생 목록 이름 저장할 리스트
    vedio_name_list = [] # 영상 이름 저장할 리스트
    view_count = [] # 조회수 저장할 리스트
    like_count = [] # 좋아요 수 저장할 리스트
        
    vedio_names = playlist.find_all("a") # 영상 이름
    for vedio_name in vedio_names:
        vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")
        if (vedio_name_tmp != None):
            vedio_name_tmp = vedio_name.select_one("dl dt a tooltip")['title'].strip()
            vedio_name_list.append(vedio_name_tmp)
    
    name_tmp = playlist_names[i]
    view_results = playlist.find_all('span', 'hit') # 조회수들 가져옴
    for view_result in view_results:
        view_result = view_result.get_text()
        view = int(view_result[4:].replace(',', '')) # 조회수 글자제외, int형 변환
        view_count.append(view) # 리스트에 조회수 저장
        playlist_name_list.append(name_tmp)
            
    like_results = playlist.find_all('span', 'like') # 좋아요 수 가져옴
    for like_result in like_results:
        like_result = like_result.get_text().strip()
        like = int(like_result[5:].replace(',', '')) # 좋아요 수 글자제외, int형 변환
        like_count.append(like) # 리스트에 좋아요 수 저장
    
    # 데이터 프레임 생성
    df = pd.DataFrame([x for x in zip(playlist_name_list, vedio_name_list, view_count, like_count)],
                        columns=("회차", "영상 이름", "조회수", "좋아요 수"))
    # 앞에서 만들어 둔 데이터 프레임에 추가
    superband_view = superband_view.append(df)

In [34]:
superband_view

,회차,영상 이름,조회수,좋아요 수
0,슈퍼밴드2 14회,[파이널] TOP6 등장! 제 2대 슈퍼밴드의 주인공은?! | JTBC 211004 방송,9005,26
1,슈퍼밴드2 14회,＂보여드리겠습니다＂ 개성 뚜렷한 TOP6의 뜨거운 각오! | JTBC 211004 방송,5155,25
2,슈퍼밴드2 14회,＂1등 못하면 억울할 것 같아요( •̀ ω •́ )✧＂ 승부욕으로 똘똘 뭉친 카디🔥...,4327,66
3,슈퍼밴드2 14회,온몸이 들썩😆 희망 가득한 카디의 자작곡 〈Ready to Fly〉♬ | JTBC ...,22094,568
4,슈퍼밴드2 14회,"🎉축하🎉 유일무이 국악 파이널리스트 박다울, 생일 선물로 우승 가보자고↗ | JTB...",7161,92
...,...,...,...,...
22,슈퍼밴드2 1회,"피처링 러브콜 1순위✨ R&B 신예, 대세 보컬리스트 문수진 | JTBC 21062...",9111,25
23,슈퍼밴드2 1회,음색 깡패♨ 밴드 보컬로 도전하는 문수진의 〈Set Fire to the Rain〉...,34995,337
24,슈퍼밴드2 1회,[단독 선공개] 어느 쪽을 보고 노래를..💦 생애 첫 무대인 데미안의 〈Holy〉♬...,68466,461
25,슈퍼밴드2 1회,슈퍼밴드2 2회 예고편,20897,45


In [35]:
# csv 파일로 저장
# 2021-11-17 11:31 저장
superband_view.to_csv('superband_view_name.csv', index=False, encoding="utf-8-sig")

In [36]:
save_test = pd.read_csv('superband_view_name.csv')
save_test

,회차,영상 이름,조회수,좋아요 수
0,슈퍼밴드2 14회,[파이널] TOP6 등장! 제 2대 슈퍼밴드의 주인공은?! | JTBC 211004 방송,9005,26
1,슈퍼밴드2 14회,＂보여드리겠습니다＂ 개성 뚜렷한 TOP6의 뜨거운 각오! | JTBC 211004 방송,5155,25
2,슈퍼밴드2 14회,＂1등 못하면 억울할 것 같아요( •̀ ω •́ )✧＂ 승부욕으로 똘똘 뭉친 카디🔥...,4327,66
3,슈퍼밴드2 14회,온몸이 들썩😆 희망 가득한 카디의 자작곡 〈Ready to Fly〉♬ | JTBC ...,22094,568
4,슈퍼밴드2 14회,"🎉축하🎉 유일무이 국악 파이널리스트 박다울, 생일 선물로 우승 가보자고↗ | JTB...",7161,92
...,...,...,...,...
455,슈퍼밴드2 1회,"피처링 러브콜 1순위✨ R&B 신예, 대세 보컬리스트 문수진 | JTBC 21062...",9111,25
456,슈퍼밴드2 1회,음색 깡패♨ 밴드 보컬로 도전하는 문수진의 〈Set Fire to the Rain〉...,34995,337
457,슈퍼밴드2 1회,[단독 선공개] 어느 쪽을 보고 노래를..💦 생애 첫 무대인 데미안의 〈Holy〉♬...,68466,461
458,슈퍼밴드2 1회,슈퍼밴드2 2회 예고편,20897,45


In [37]:
save_test.groupby('회차').count()

,영상 이름,조회수,좋아요 수
회차,,,
슈퍼밴드2 10회,29,29,29
슈퍼밴드2 11회,32,32,32
슈퍼밴드2 12회,33,33,33
슈퍼밴드2 13회,40,40,40
슈퍼밴드2 14회,27,27,27
슈퍼밴드2 1회,27,27,27
슈퍼밴드2 2회,26,26,26
슈퍼밴드2 3회,44,44,44
슈퍼밴드2 4회,32,32,32
